In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
# 获取样本数据
train_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
train_data

In [ ]:
# 获取预测数据
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test_data

In [ ]:
# 给预测数据补充一列
test_data.insert(13,'Transported',False)
test_data

In [ ]:
# 将数据拼接一起
data = train_data.append(test_data,ignore_index = True)
data

In [ ]:
# 查看数据详情
data.info()

In [ ]:
# 查看HomePlanet 
data['HomePlanet'].value_counts()

In [ ]:
# 将缺失值进行填充
data['HomePlanet'] = data['HomePlanet'].fillna('Earth')
data['HomePlanet'].value_counts()

In [ ]:
# 查看乘客休眠状态
data['CryoSleep'].value_counts()

In [ ]:
# 将缺失的休眠状态进行填充
data['CryoSleep'] = data['CryoSleep'].fillna('False')
data.info()

In [ ]:
# 查看仓号
data['Cabin'].value_counts()

In [ ]:
data['Cabin'] = data['Cabin'].fillna('G/734/S')
data.info()

In [ ]:
# 将仓号按照P,S进行区别
def func(name):
    str1 = name.split('/')[-1]
    str2 = str1.strip()
    return str2
data['Cabin'] = data['Cabin'].map(func)
data['Cabin'].value_counts()

In [ ]:
# 查看乘客从那个星球下船
data['Destination'].value_counts()

In [ ]:
# 填补缺失值
data['Destination'] = data['Destination'].fillna('TRAPPIST-1e')
data.info()

In [ ]:
# 年龄用中位数进行填充
data['Age'] = data['Age'].fillna(data['Age'].median())
data.info()

In [ ]:
# 查看vip分布数值
data['VIP'].value_counts()

In [ ]:
# 用False进行填充
data['VIP'] = data['VIP'].fillna('False')
data.info()

In [ ]:
# 查看乘客花费金额
data['RoomService'].value_counts()

In [ ]:
# 用均值进行填充
data['RoomService'] = data['RoomService'].fillna(data['RoomService'].mean())
data.info()

In [ ]:
# 查看FoodCourt
data['FoodCourt'].value_counts()

In [ ]:
# 用均值进行填充
data['FoodCourt'] = data['FoodCourt'].fillna(data['FoodCourt'].mean())
data.info()

In [ ]:
data['ShoppingMall'] = data['ShoppingMall'].fillna(data['ShoppingMall'].mean())
data['Spa'] = data['Spa'].fillna(data['Spa'].mean())
data['VRDeck'] = data['VRDeck'].fillna(data['VRDeck'].mean())
data.info()

In [ ]:
# 把名字这列删除
data = data.drop(['Name'],axis=1)
data.head()

In [ ]:
homePlanet = pd.get_dummies( data['HomePlanet'] , prefix='HomePlanet' )
homePlanet

In [ ]:
# 数据拼接
data = pd.concat([data,homePlanet],axis=1)
data.head()

In [ ]:
data['CryoSleep'] = data['CryoSleep'].map(lambda x : 1 if x==True else 0)
data.head()

In [ ]:
cabin = pd.get_dummies( data['Cabin'] , prefix='Cabin' )
cabin.head()

In [ ]:
data = pd.concat([data,cabin],axis=1)
data.head()

In [ ]:
destination = pd.get_dummies( data['Destination'] , prefix='Destination' )
destination

In [ ]:
data = pd.concat([data,destination],axis=1)
data.head()

In [ ]:
data = data.drop(['HomePlanet','Cabin','Destination'],axis=1)
data.head()

In [ ]:
# 将年龄进行标准化处理
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data['Age'] = scaler.fit_transform(data['Age'].values.reshape(-1,1))
data.head()

In [ ]:
# 对VIP进行转化，化为0-1的形式
data['VIP'] = data['VIP'].map(lambda x : 1 if x==True else 0)
data.head()

In [ ]:
# 对RoomService进行标准化处理
data['RoomService'] = scaler.fit_transform(data['RoomService'].values.reshape(-1,1))
data.head()

In [ ]:
# 对FoodCourt进行标准化处理
data['FoodCourt'] = scaler.fit_transform(data['FoodCourt'].values.reshape(-1,1))
data['ShoppingMall'] = scaler.fit_transform(data['ShoppingMall'].values.reshape(-1,1))
data['Spa'] = scaler.fit_transform(data['Spa'].values.reshape(-1,1))
data['VRDeck'] = scaler.fit_transform(data['VRDeck'].values.reshape(-1,1))
data.head()

In [ ]:
data['Transported'] = data['Transported'].map(lambda x : 1 if x==True else 0)
data.head()

In [ ]:
# 获取训练数据
sample_data = data.loc[:8692,:]
sample_data

In [ ]:
# 获取特征数据和标签数据
x_data = sample_data.drop(['PassengerId','Transported'],axis=1)
y_data = sample_data['Transported']
x_data

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
ploy_feature = PolynomialFeatures(degree=3)
# 选取部分特征进行预测
label = ['Transported']
x_train_ploy = ploy_feature.fit_transform(x_data)
# 生成逻辑回归模型
lr_test = LogisticRegression()
tree_param_grid = {'penalty' : ['l1'],'C':[0.03,0.1,0.3], 'solver' : ['liblinear']}
grid = GridSearchCV(lr_test,param_grid=tree_param_grid,cv=5)
grid.fit(x_train_ploy,y_data)
grid.cv_results_,grid.best_params_,grid.best_score_

In [ ]:
# 获取处理后的测试数据
test_datas = data.loc[8693:,:]
test_datas

In [ ]:
# 保存文件的表格形式
pre_data = test_datas.loc[:,['PassengerId','Transported']]
pre_data

In [ ]:
model = LogisticRegression(**{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'})
model.fit(x_train_ploy,y_data)
z_data = test_datas.drop(['PassengerId','Transported'],axis=1)
z_data_x = ploy_feature.fit_transform(z_data)
pre_data['Transported'] = model.predict(z_data_x)
pre_data['Transported'].value_counts()

In [ ]:
pre_data['Transported'] = pre_data['Transported'].map(lambda x : True if x == 1 else False)
pre_data['Transported'].value_counts()

In [ ]:
pre_data.to_csv('submission.csv',index=False)  